<img src="../docs/assets/JobShopLabLogo.svg" width="50" align="left" style="margin-right: 10px;">

# Stochastic behavior in JobShopLab


In [2]:
from jobshoplab.types.stochasticy_models import *
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.express as px
from jobshoplab.utils.load_config import load_config
from jobshoplab.compiler.repos import DslStrRepository
from jobshoplab.compiler import Compiler
from jobshoplab import JobShopLabEnv
from jupyter_utils import change_to_jobshoplab
from pathlib import Path
change_to_jobshoplab()

C:\Users\klno603\Anaconda3\envs\jsl\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


Already in the desired directory: C:\Users\klno603\PycharmProjects\jobshoplab_


## Distribution Explorer

This interactive tool lets you explore four different probability distributions. Each time you select one, a sample of random values is drawn and visualized. Here's a brief explanation of each:

* **Poisson Distribution**
    Models the number of events occurring in a fixed interval of time or space.
    Example: Number of emails received per hour.
    Shape: Discrete, skewed right for small means.

* **Gaussian (Normal) Distribution**
    The classic bell curve. Models many natural phenomena due to the Central Limit Theorem.
    Example: Heights of people, measurement errors.
    Shape: Symmetric and continuous.

* **Uniform Distribution**
    Uniform distribution assigns equal probability to all values within a specified range.  
    Example: Rolling a fair die.  
    Shape: Flat and continuous.

* **Gamma Distribution**
    Models waiting times or the sum of multiple exponential variables.
    Example: Time until the k-th event in a Poisson process.
    Shape: Skewed continuous distribution.

Each distribution object in this notebook has a .update() method that generates a new random value, which is stored in .time. The histogram shows how these values are distributed over many samples.

In [32]:
# available distributions and their parameters
#! change the values here to observe the changes in the plots
possion = PoissonFunction(base_time=1)
gauss = GaussianFunction(base_time=5, std=1)
uniform = UniformFunction(base_time=10, offset=2)
gamma = GammaFunction(base_time=5.e4, shape=1., scale=10000.)

# Sample size
sample_size = 1000

# Assuming these distribution objects are already defined
# possion, gauss, gamma

dist_mapping = {
    "Gamma": gamma,
    "Poisson": possion,
    "Gaussian": gauss,
    "Uni": uniform,
}

def get_sample(distribution):
    distribution.update()
    return distribution.time

def make_plot(distribution, sample_size):
    samples = [get_sample(distribution) for _ in range(sample_size)]
    fig = px.histogram(
        x=samples,
        marginal="box",
        title=f"Distribution: {distribution.__class__.__name__}",
        labels={"x": "Sampled Values", "y": "Time Value"},
    )
    fig.show()

# Dropdown widget
dropdown = widgets.Dropdown(
    options=list(dist_mapping.keys()),
    value="Gamma",
    description="Distribution:"
)

# Output widget to contain the plot
output = widgets.Output()

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        with output:
            clear_output(wait=True)
            dist_obj = dist_mapping[change['new']]
            make_plot(dist_obj, sample_size)

dropdown.observe(on_change)

# Display widgets and initial plot
display(dropdown, output)

with output:
    make_plot(dist_mapping[dropdown.value], sample_size)



Dropdown(description='Distribution:', options=('Gamma', 'Poisson', 'Gaussian', 'Uni'), value='Gamma')

Output()

# Setting stochasitiy for the Instance

in order to use stochastic times this behavior needs to be defined somehow. 

Every Deffinition (Duration or Frequency) is settable

**Example of how to set the time behavior for the setup times**

```yaml
# Stochastic Configuration
setup_times:
    - machine: "m-0"
      specification: |
        tl-0|tl-1|tl-2
        tl-0|0 2 5
        tl-1|2 0 8
        tl-2|5 2 0
      time_behavior: static # Static setup time 
          
    - machine: "m-1"
      specification: |
        tl-0|tl-1|tl-2
        tl-0|0 2 5
        tl-1|2 0 8
        tl-2|5 2 0
      time_behavior: # Stochastic setup time Note: simple pass type and the arguments of the TimeModel
        type: "uniform"
        offset: 2

    - machine: "m-2" # also static here
      specification: |
        tl-0|tl-1|tl-2
        tl-0|0 2 5
        tl-1|2 0 8
        tl-2|5 2 0
```

**resulting mapping**

In [1]:
instance = """
title: InstanceConfig

# Example of a 3x3 Instance
# with AGVs and outages and setup times and stochastic times

instance_config:
  description: "example config"
  instance:
    description: "3x3"
    specification: |
      (m0,t)|(m1,t)|(m2,t)
      j0|(0,3) (1,2) (2,2)
      j1|(0,2) (2,1) (1,4)
      j2|(1,4) (2,3) (0,3)
    time_behavior: static
    tool_usage:
      - job: "j0"
        operation_tools: ["tl-0", "tl-1", "tl-2"]
      - job: "j1"
        operation_tools: ["tl-0", "tl-1", "tl-2"]
      - job: "j2"
        operation_tools: ["tl-0", "tl-1", "tl-2"]
  
  setup_times:
    - machine: "m-0"
      specification: |
        tl-0|tl-1|tl-2
        tl-0|0 2 5
        tl-1|2 0 8
        tl-2|5 2 0
      time_behavior: static
          
    - machine: "m-1"
      specification: |
        tl-0|tl-1|tl-2
        tl-0|0 2 5
        tl-1|2 0 8
        tl-2|5 2 0
      time_behavior:
        type: "uniform"
        offset: 2

    - machine: "m-2"
      specification: |
        tl-0|tl-1|tl-2
        tl-0|0 2 5
        tl-1|2 0 8
        tl-2|5 2 0

      time_behavior:
        type: "gaussian"
        std: 1


  logistics: 
    type: "agv"
    amount: 3
    specification: |
      m-0|m-1|m-2|in-buf|out-buf
      m-0|0 2 5 2 7
      m-1|2 0 8 3 6
      m-2|5 2 0 6 2
      in-buf|2 3 6 0 9
      out-buf|7 5 2 9 0
    
    time_behavior:
      type: "poisson"
  
  outages:
    - component: "m"
      type: "maintenance"
      duration: 5
      frequency: 
        type: "gamma"
        scale: 5
        base: 10

    - component: "t"
      type: "recharge"
      duration: 
        type: "gaussian"
        std: 1
        base: 10
      frequency: 10

        

init_state:
  t-0:
    location: "m-1"
  t-1:
    location: "m-2"
  t-2:
    location: "m-2"
"""

In [5]:
config = load_config(config_path = Path("./data/config/getting_started_config.yaml"))
repo = DslStrRepository(instance,1,config)
compiler = Compiler(config, loglevel="warning", repo=repo)
env = JobShopLabEnv(config=config, compiler=compiler)

# show the setuptime mapping
env.instance.machines[1].setup_times

{('tl-0', 'tl-0'): UniformFunction(base_time=0, low=-2.0, high=2.0),
 ('tl-0', 'tl-1'): UniformFunction(base_time=2, low=0.0, high=4.0),
 ('tl-0', 'tl-2'): UniformFunction(base_time=5, low=3.0, high=7.0),
 ('tl-1', 'tl-0'): UniformFunction(base_time=2, low=0.0, high=4.0),
 ('tl-1', 'tl-1'): UniformFunction(base_time=0, low=-2.0, high=2.0),
 ('tl-1', 'tl-2'): UniformFunction(base_time=8, low=6.0, high=10.0),
 ('tl-2', 'tl-0'): UniformFunction(base_time=5, low=3.0, high=7.0),
 ('tl-2', 'tl-1'): UniformFunction(base_time=2, low=0.0, high=4.0),
 ('tl-2', 'tl-2'): UniformFunction(base_time=0, low=-2.0, high=2.0)}

In [11]:
# access the setup time for a specific tool
print("init time:" ,env.instance.machines[1].setup_times[('tl-0', 'tl-1')].time)

# sample a new time 
env.instance.machines[1].setup_times[('tl-0', 'tl-1')].update()
print("after_update: ",env.instance.machines[1].setup_times[('tl-0', 'tl-1')].time)

init time: 2
after_update:  2
